# Getting Started

## Install the SingleStore package

The SingleStore package can be installed the following way:
```
pip install singlestoredb
```

In addition, you can install the SQLAlchemy and Ibis plugins with the following:
```
pip install singlestore[dataframe]
```

## Import SingleStore

In [1]:
import singlestoredb as s2

## Create a connection

This function is compliant with the Python DB-API 2.0. In addition, it allows you to use a URL
as a connection string rather than individual parameters. Parameters may also be set using
environment variable (e.g., `SINGLESTOREDB_HOST`, `SINGLESTOREDB_USER`, `SINGLESTOREDB_PASSWORD`, etc.),
but it is easiest to specify the entire URL in the `SINGLESTOREDB_URL` environment variable.

In [2]:
conn_url = 'root:@127.0.0.1/x_db'

In [3]:
conn = s2.connect(conn_url)

The URL in the code above can take a couple of forms. The default driver is `mysql`, but you can also specify `http` for connecting to the SingleStoreDB data API.
```
http://root:@localhost:9000/x_db
```

### Environment Variables

Connection URLs can also be set using an environment variable. This allows you to create environments that already have connection parameters
embedded in them so that new connections can be made without specifying any parameters.

In [4]:
import os

os.environ['SINGLESTOREDB_URL'] = conn_url

### Cursors

Most interaction with databases are done using cursors. To create a cursor from a connection, you use the `cursor` method.

In [5]:
cur = conn.cursor()

## Create tables and insert data

Using the cursor, we can execute commands for creating tables and inserting data.

### Reset Tables

In [6]:
cur.execute(r'drop table if exists departments')
cur.execute(r'drop table if exists employees')
cur.execute(r'drop table if exists salaries')

cur.execute(r'''
create table if not exists departments (
  id int,
  name varchar(255),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists employees (
  id int,
  deptId int,
  managerId int,
  name varchar(255),
  hireDate date,
  state char(2),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists salaries (
  employeeId int,
  salary int,
  primary key (employeeId)
);''')

0

### Insert Data Using Named Parameters

Using named parameters such as `:foo` is the default parameter format in queries. This requires dictionaries for the parameter structure.

In [7]:
cur.executemany(r'insert into departments(id, name) values (%(id)s, %(name)s)',
    [
        dict(id=1, name='Marketing'),
        dict(id=2, name='Finance'),
        dict(id=3, name='Sales'),
        dict(id=4, name='Customer Service'),
    ]
)

4

### Insert Data Using Positional Parameters

You can also use positional parameters which specify the one-based index in a list or tuple.

In [8]:
cur.executemany(r'insert into employees (id, deptId, managerId, name, hireDate, state) '
                r'values (%s, %s, %s, %s, %s, %s)',
    [
        (1, 2, None, "Karly Steele", "2011-08-25", "NY"),
        (2, 1, 1, "Rhona Nichols", "2008-09-11", "TX"),
        (3, 4, 2, "Hedda Kent", "2005-10-27", "TX"),
        (4, 2, 1, "Orli Strong", "2001-07-01", "NY"),
        (5, 1, 1, "Leonard Haynes", "2011-05-30", "MS"),
        (6, 1, 5, "Colette Payne", "2002-10-22", "MS"),
        (7, 3, 4, "Cooper Hatfield", "2010-08-19", "NY"),
        (8, 2, 4, "Timothy Battle", "2001-01-21", "NY"),
        (9, 3, 1, "Doris Munoz", "2008-10-22", "NY"),
        (10, 4, 2, "Alea Wiggins", "2007-08-21", "TX"),
    ]
)

10

### Insert Data From DataFrame

In addition to this method, you can use the `pandas.DataFrame.to_sql` method with a SQLAlchemy SingleStore connection object.

In [9]:
import pandas as pd

df = pd.DataFrame( 
    [
        (1, 885219), 
        (2, 451519), 
        (3, 288905), 
        (4, 904312), 
        (5, 919124),
        (6, 101538), 
        (7, 355077), 
        (8, 900436), 
        (9, 41557), 
        (10, 556263),
    ],
    columns=['employeeId', 'salary'])

cur.executemany(r'insert into salaries (employeeId, salary) '
                r'values (%s, %s)', df)

10

# Querying data

Here we are executing a simple `SELECT` operation.

In [10]:
cur.execute('select name from employees')

10

There are various ways of fetching data including the `fetchone()`, `fetchmany()`, and `fetchall()` methods
description in the DB-API. In addition, you can simply iterate over the cursor itself.

In [11]:
for item in cur:
    print(item)

('Karly Steele',)
('Rhona Nichols',)
('Hedda Kent',)
('Orli Strong',)
('Cooper Hatfield',)
('Doris Munoz',)
('Leonard Haynes',)
('Colette Payne',)
('Timothy Battle',)
('Alea Wiggins',)


By default, results come back in tuples, but there are other result structure options as well, they include `namedtuple` and `dict`.

In [12]:
conn = s2.connect(conn_url, results_type='dicts')

In [13]:
cur = conn.cursor()
cur.execute('select name from employees')
cur.fetchall()

[{'name': 'Karly Steele'},
 {'name': 'Rhona Nichols'},
 {'name': 'Hedda Kent'},
 {'name': 'Orli Strong'},
 {'name': 'Cooper Hatfield'},
 {'name': 'Doris Munoz'},
 {'name': 'Timothy Battle'},
 {'name': 'Alea Wiggins'},
 {'name': 'Leonard Haynes'},
 {'name': 'Colette Payne'}]

The following is a query using parameter substitution. While the underlying connection object is a `mysql.connector`-based
object which takes the pyformat parameter format, we are using the `sqlparams` package to allow us to 
use whatever substitution form we want. Since the HTTP API uses question marks for parameters, we have
adopted that format here as well.

In [14]:
cur.execute('select name, hireDate from employees where name like %s', ['%Rhona%'])

1

The `description` field returns information about the query results. It includes fields such as `name`, `type_code`,
`display_size`, `internal_size`, etc. These are defined in the DB-API, but are not all populated.

In [15]:
cur.description

(Description(name='name', type_code=253, display_size=None, internal_size=255, precision=255, scale=0, null_ok=True, flags=0, charset=33),
 Description(name='hireDate', type_code=10, display_size=None, internal_size=30, precision=30, scale=31, null_ok=True, flags=0, charset=33))

In [16]:
cur.fetchall()

[{'name': 'Rhona Nichols', 'hireDate': datetime.date(2008, 9, 11)}]

## Server Variables

Server variables can be accessed through a number of dictionary-like members on the connection. The members are `globals`, `locals`, `cluster_globals`, `cluster_locals` when a scope is specified, or `vars` and `cluster_vars` for all variables regardless of scope.

In [17]:
dict(conn.globals)

{'active_query_users_blacklist': '',
 'activities_delta_sleep_s': '1',
 'advanced_hdfs_pipelines': False,
 'aes_default_encryption_mode': 'aes-128-ecb',
 'aggregator_failure_detection': False,
 'allow_async_bottomless': False,
 'allow_newer_to_older_replication': False,
 'allow_proc_cpuinfo_errors': False,
 'assert_on_cluster_db_reprovisioning': False,
 'async_backup': True,
 'auditlog_disk_sync': False,
 'auditlog_level': False,
 'auditlog_retention_period': '0',
 'auditlog_rotation_size': '134217728',
 'auditlog_rotation_time': '3600',
 'auto_replicate': False,
 'autocommit': True,
 'backup_max_threads': '0',
 'basedir': '/home/ksmith/memsql/release',
 'batch_external_functions': 'ALWAYS',
 'batch_external_functions_size': '512',
 'blob_cache_eviction_policy': 'LRU2',
 'blob_cache_threadpool_max_concurrent': '32',
 'bottomless_blob_compression_level': '0',
 'bottomless_compression_level': '1',
 'bottomless_gc_retention_period_minutes': '1440',
 'bottomless_gc_retention_period_test_mo

In [18]:
conn.globals.enable_external_functions = True

In [19]:
conn.globals.enable_external_functions

True

In [20]:
cur.execute('show variables like "enable_external_functions"')
cur.fetchall()

[{'Variable_name': 'enable_external_functions', 'Value': 'ON'}]

## Enabling the HTTP API

The HTTP API can be enabled using the `conn.enable_http_api` method. This method can optionally set the port number.
If a port number is not specified, the existing setting will be used. The port number is returned.

In [21]:
conn.enable_http_api(port=8100)

8100

## Create an HTTP connection

In [22]:
s2.options.results.type = 'namedtuples'

In [23]:
http_conn = s2.connect('http://root:@localhost:8100/x_db')

In [24]:
http_cur = http_conn.cursor()

## Query data using the HTTP connection

In [25]:
http_cur.execute('select name from employees')

10

In [26]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [27]:
http_cur.fetchall()

[Row(name='Karly Steele'),
 Row(name='Rhona Nichols'),
 Row(name='Hedda Kent'),
 Row(name='Orli Strong'),
 Row(name='Cooper Hatfield'),
 Row(name='Doris Munoz'),
 Row(name='Timothy Battle'),
 Row(name='Alea Wiggins'),
 Row(name='Leonard Haynes'),
 Row(name='Colette Payne')]

In [28]:
http_cur.execute('select name, hireDate from employees where name like %s', ['%Rhona%'])

1

In [29]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0),
 Description(name='hireDate', type_code=10, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [30]:
df = http_cur.fetchall()
df

[Row(name='Rhona Nichols', hireDate=datetime.date(2008, 9, 11))]

## Workspace Management

The objects allow you to manage workspaces and create database connections to those workspaces. In order for this call
to work, you either need to pass in a cluster management API token or have one set in your `SINGLESTOREDB_CLUSTER_MANGER_TOKEN` environment variable.

In [31]:
wm = s2.manage_workspaces()

### Get Current Clusters and Regions

In [32]:
wm.workspace_groups

[WorkspaceGroup(created_at=datetime.datetime(2022, 12, 1, 0, 0), firewall_ranges=['0.0.0.0/0'], id='1b51e41d-79a9-4bbb-9343-494088fa23b5', name='Python Client Testing', region=Region(id='6be78bc7-5b35-4272-959f-8829b6f8bc63', name='US East 1 (N. Virginia)', provider='AWS'))]

In [33]:
wm.regions

[Region(id='0bbbde94-7c33-4592-95bc-470332396a82', name='Asia Southeast 1 (Singapore)', provider='GCP'),
 Region(id='1c1de314-2cc0-4c74-bd54-5047ff90842e', name='US West 1 (Oregon)', provider='GCP'),
 Region(id='22a913e1-ace5-4362-afa0-3d21a01d6d1c', name='US Central 1 (Iowa)', provider='GCP'),
 Region(id='3186a27b-87d5-4b84-a93b-90c57fd2be74', name='Asia South 1 (Mumbai)', provider='GCP'),
 Region(id='35551562-6d39-4e14-8747-0f42114a330b', name='Europe West 1 (Ireland)', provider='AWS'),
 Region(id='3d226d4b-90b4-4a8d-848c-6c8403ee905c', name='Asia Pacific Southeast 1 (Singapore)', provider='AWS'),
 Region(id='3de9e276-1c93-4987-9463-5700aa08a837', name='US West 2 (Oregon)', provider='AWS'),
 Region(id='57ba8143-d4cc-470a-901f-871c684ee2bc', name='Europe North 1 (Stockholm)', provider='AWS'),
 Region(id='6be78bc7-5b35-4272-959f-8829b6f8bc63', name='US East 1 (N. Virginia)', provider='AWS'),
 Region(id='6e1b6fae-fe87-4aaa-bc6e-55d9e4c23806', name='Asia Southeast 2 (Jakarta)', provider=

### Create a Workspace Group

In [34]:
import secrets
password = secrets.token_urlsafe(20)

In [36]:
wg = wm.create_workspace_group(
    'Demo Workspace Group', 
    region=[x for x in wm.regions if x.name.startswith('US')][0],
    admin_password=password, 
    firewall_ranges=['0.0.0.0/0'],
)
wg

WorkspaceGroup(created_at=datetime.datetime(2022, 12, 7, 0, 0), id='6dba376f-4d25-4c9b-a2f0-31f2bd7d1af9', name='Demo Workspace Group', region=Region(id='1c1de314-2cc0-4c74-bd54-5047ff90842e', name='US West 1 (Oregon)', provider='GCP'))

### Create a Workspace in the Group

In [37]:
ws = wg.create_workspace('workspace-1', wait_on_active=True)

In [38]:
ws

Workspace(created_at=datetime.datetime(2022, 12, 7, 0, 0), endpoint='svc-6dba376f-4d25-4c9b-8f70-f69608be78c7-dml.gcp-oregon-1.svc.singlestore.com', group_id='6dba376f-4d25-4c9b-a2f0-31f2bd7d1af9', id='7936231a-dc53-4a83-8f70-f69608be78c7', name='workspace-1', size='S-00', state='ACTIVE')

### Connect to the Workspace

In [39]:
with ws.connect(user='admin', password=password) as conn:
    with conn.cursor() as cur:
        cur.execute('show databases')
        print(cur.fetchall())

[singlestoredb.Row(Database='cluster'), singlestoredb.Row(Database='information_schema'), singlestoredb.Row(Database='memsql')]


In [40]:
ws.terminate(wait_on_terminated=True)
wg.terminate()